In [ ]:
%pip install yfiles_jupyter_graphs
%pip install pandas
%pip install requests
%pip install termcolor
%pip install openpyxl
%pip install lxml
%pip install blockcypher

In [6]:
from yfiles_jupyter_graphs import GraphWidget
from termcolor import colored
import requests
from bs4 import BeautifulSoup
import re
import pandas
import numpy
import blockcypher as bc

In [7]:
def buildgraph(addr, addrlist):
    w = GraphWidget()
    
    nodelist = [{"id": 0, "properties": {"label": addr}}]
    edgelist = []
    i = 0
    for address in addrlist:
        i += 1
        nodelist.append({"id": i, "properties": {"label": address, "Transactions": addrlist[address]}})
        edgelist.append({"id": i, "start": 0, "end": i, "properties": {"label": "knows"}})
    
    w.nodes = nodelist
    w.edges = edgelist
    w.directed = True
    w.show()

In [8]:
def listtodict(addrlist):
    addrdict = {}
    for i in addrs:
        if addrs.count(i) > 1:
            addrdict[i] = addrs.count(i)
    return addrdict

In [1]:
def currencyTX(addr, maximum, coin):
    coin = coin.upper()
    match coin:
        case 'BTC':
            from addressdata import gettransactions
            from addressdata import accounttransactions
            r = gettransactions(addr, maximum)
            list = accounttransactions(r, maximum, addr)
        case 'ETH':
            from Etherscan import gettransactions
            gettransactions(addr, maximum)
            list = gettransactions.addr_list
        case _:
            print('Error')
            list = []
    return list

def transactions(address, coin, end):
    # still collect data for eth and btc from respective apis
    # able to collect more transactions in less api calls
    if coin == 'ETH' or coin == 'BTC':
        addrlist = currencyTX(address, end, coin)
    else:
        from BlockCypher import addrfull
        addrfull(address, coin, 'USD', end)
        addrlist = addrfull.addr_list
    
    export = input(colored('\nWould you like the transaction data exported to a graph? (Y/N):\n', 'blue'))

    if export.upper() == "Y" or export.upper() == "YES":
        addrs = listtodict(addrlist)
        buildgraph(address, addrs)

def findaccount():
    from account import choosecurrency
    coin = choosecurrency()
    from account import account
    addr = account(coin)
    accountmenu(addr, coin)

def accountmenu(address, coin):
    print(colored('\nWhat would you like to do next?', 'green'))
    print('1. Search for this account online')
    print('2. Gather this account\'s transactions')
    print('3. Find a different account')
    print('4. Main Menu')
    print('5. Quit')
    choice = input(colored('\nPlease enter your choice (1-5):\n', 'blue'))

    if choice == '1':
        print(address)
        URL = 'https://www.allprivatekeys.com/btc/' + address
        r = requests.get(URL)
        soup = BeautifulSoup(r.content, 'html5lib')
        acctInfo = str(soup.find_all('div', attrs={'class': 'media text-muted pt-3'}))
        potentialLink = (re.findall(r'(https?://\S+)', acctInfo)) + (re.findall(r'(https?://\S+)', acctInfo))
        if not potentialLink:
            print('Sorry, we couldn\'t find the account mentioned anywhere...')
        else:
            potentialLinkstr = ''
            for i in potentialLink:
                potentialLinkstr += i
            potentialLinkstr = potentialLinkstr.replace("</a><br/>", ' ')
            potentialLinkstr = potentialLinkstr.replace('"', ' ')

            # Make List and Display Search Results
            potentialLink = list(potentialLinkstr.split(' '))
            print('\nResults:\n \n', potentialLink)

            # Export as CSV
            export = input(colored('\nWould you like this exported as a csv file? (Y/N):\n', 'blue'))
            if export.upper() == "Y" or export.upper == "YES":
                dict = {'Bitcoin address found on the following sites:': potentialLink}
                df = pandas.DataFrame(dict)
                df.to_csv(address + '.csv')
                print(
                    '\nYour potential links can be found in the same folder in which you saved this program.\n \nThe file is named:\n',
                    address + '.csv')
                print(colored(
                    '\nBe Sure to note the poster\'s account or username, \nthey may be the one who owns the bitcoin account',
                    'red'))

        accountmenu(address, coin)

    elif choice == '2':

        end = input(
             colored('\nPlease type in the maximum number of transactions you would like to load.\n', 'blue'))

        print(colored('\nFetching your results. Please wait...', 'yellow'))
        transactions(address, coin, end)

    elif choice == '3':
        findaccount()

    elif choice == '4':
        main()

    elif choice == '5':
        quit()

    else:
        print('\n \nSorry, that wasn\'t a choice. Please try again')
        accountmenu(address, coin)

    accountmenu(address, coin)

In [10]:
def welcome():
    logo = colored('█████████', 'green')
    logo1 = colored('█▄─▀█▀─▄█', 'green')
    logo2 = colored('██─█▄█─██', 'green')
    logo3 = colored('▀▄▄▄▀▄▄▄▀', 'green')
    logo = logo.center(70, ' ')
    logo1 = logo1.center(70,' ')
    logo2 = logo2.center(70,' ')
    logo3 = logo3.center(70,' ')
    print(colored('This project was completed in partial fulfilment of the requirements \nfor a masters degree in Cybersecurity at Mercyhurst University.\n', 'green'))
    print(logo)
    print(logo1)
    print(logo2)
    print(logo3)
    print(colored('\nWelcome to the Bitcoin analyzer. This project is capable of finding \nbitcoin accounts associated with specific names or emails, and \nmapping transactions related to those accounts.', 'green'))

def main():
    print(colored('\nPlease select an option from the list below:', 'green'))
    print(colored('1. Find an account based on a person\'s username using Sherlock', 'red'))
    print(colored('2. Find an account based on a person\'s email', 'yellow'))
    print(colored('3. Analyze an account you already know', 'yellow'))
    print('4. Quit\n')
    
    from main import getsdn
    getsdn()

    choice = int(input((colored('Enter your choice: \n', 'blue'))))

    if choice == 1:
        user = input(colored('\nPlease enter the username you believe is linked to a bitcoin account:\n', 'blue'))
        print('')
        
#Enter path to sherlock.py here
        script_descriptor = open('sherlock/sherlock/sherlock.py')
        
        sherlock = script_descriptor.read()
        sys.argv = [sherlock, user]
        exec(sherlock)
        
    elif choice == 2:
        from EmailSearch import acctEmail
        acctEmail()

    elif choice == 3:
        findaccount()
        

    elif choice == 4:
        quit()

In [ ]:
welcome()
main()